In [2]:
import pandas as pd
df = pd.read_csv('complex_data.csv',sep='\t')
df.columns

Index(['Unnamed: 0', 'Log price', 'Log area (square meters)',
       'Distance to center 2', 'Distance to center 4', 'Log WC', 'Log Floors'],
      dtype='object')

In [3]:
df.drop("Unnamed: 0",axis=1,inplace=True)

In [4]:
df

,Log price,Log area (square meters),Distance to center 2,Distance to center 4,Log WC,Log Floors
0,-0.530310,-1.288856,0.172328,1.388260,0.323821,1.033543
1,1.518783,-0.637139,-4.732391,-0.330580,-1.312694,-0.679309
2,-0.767807,0.330336,0.192896,-0.764218,0.323821,-0.679309
3,-0.130805,-1.349877,0.279746,-0.718391,-1.312694,-0.876262
4,-0.099847,-0.362392,0.316293,-0.713225,-1.312694,-0.679309
...,...,...,...,...,...,...
1543,-0.085975,2.094927,0.197637,-0.764708,0.323821,-0.679309
1544,-1.668046,-2.104437,0.254664,-0.731946,-0.355394,0.078640
1545,-0.276697,0.735514,0.227416,-0.763698,0.323821,-0.679309
1546,-0.441114,-0.814508,0.128215,1.372125,-0.355394,0.616414


In [5]:
y = df['Log price']
X = df.drop("Log price",axis=1)

In [6]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()

regressor.fit(X,y)

from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(regressor.get_params())

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [7]:
import numpy as np
# Define a function for model evaluation using cross validation
def evaluate_model_cross_validation(name, model, X_train, y_train, folds = 10):

    from sklearn.model_selection import cross_val_score
 
    # Cross Validation Regression MAE
    metric='neg_mean_absolute_error'
    scores = cross_val_score(regressor, X_train, y_train, scoring=metric, cv=folds, n_jobs=-1)
    mean_score, std_score = np.mean(scores), np.std(scores)
    print('>%s - training - MAE: %.3f (+/-%.3f)' % (name, mean_score, std_score))
    
    
    # Cross Validation Regression MSE
    metric='neg_mean_squared_error'
    scores = cross_val_score(regressor, X_train, y_train, scoring=metric, cv=folds, n_jobs=-1)
    mean_score, std_score = np.mean(scores), np.std(scores)
    print('>%s - training - MSE: %.3f (+/-%.3f)' % (name, mean_score, std_score))
    
    # Cross Validation Regression R^2
    metric='r2'
    scores = cross_val_score(regressor, X_train, y_train, scoring=metric, cv=folds, n_jobs=-1)
    mean_score, std_score = np.mean(scores), np.std(scores)
    print('>%s - training - R^2: %.3f (+/-%.3f)' % (name, mean_score, std_score))

In [8]:
# Define the random grid

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [9]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [10]:
# First create the base model to tune 
regressor = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator = regressor,
                               param_distributions = random_grid,
                               n_iter = 100, cv = 5, verbose=2, random_state=42,
                               n_jobs = -1)
rf_random.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/tiamo/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
205 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
155 fits failed with the following error:
Traceback (most recent call last):
  File "/home/tiamo/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/tiamo/.local/lib/python3.10/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/home/tiamo/.local/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/tiamo/.local/l

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [11]:
rf_random.best_params_

{'n_estimators': 1200,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 20,
 'bootstrap': True}

In [12]:
# Fitting Random forest Model to the dataset using the best parameters after random search
rf_r = RandomForestRegressor(bootstrap = True,
                                  max_depth = 20,
                                  max_features ='sqrt',
                                  min_samples_leaf = 2,
                                  min_samples_split = 5,
                                  n_estimators = 1200)

rf_r.fit(X, y)
evaluate_model_cross_validation("RF_random", rf_r, X, y, 10)

>RF_random - training - MAE: -0.525 (+/-0.030)
>RF_random - training - MSE: -0.531 (+/-0.087)
>RF_random - training - R^2: 0.465 (+/-0.065)


In [13]:
# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV

# Number of trees in random forest
n_estimators = [800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [10, 20, 30, 40]
# Minimum number of samples required to split a node
min_samples_split = [5,6]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2,3]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
parameters = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

regressor = RandomForestRegressor()

grid_search = GridSearchCV(estimator = regressor,
                           param_grid = parameters,
                           cv = 5, n_jobs = -1, verbose = 2)

grid_search = grid_search.fit(X, y)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


In [14]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 30,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 6,
 'n_estimators': 1300}

In [15]:
# Fitting Random forest Model to the dataset using the best parameters
rf_g = RandomForestRegressor(bootstrap = True,
                                  max_depth = 30,
                                  max_features ='sqrt',
                                  min_samples_leaf = 2,
                                  min_samples_split = 6,
                                  n_estimators = 1300)

rf_g.fit(X, y)
evaluate_model_cross_validation("RF_grid_search", rf_g, X, y, 10)

>RF_grid_search - training - MAE: -0.528 (+/-0.031)
>RF_grid_search - training - MSE: -0.532 (+/-0.087)
>RF_grid_search - training - R^2: 0.464 (+/-0.061)
